# NER 학습 데이터 생성기

Preprocessing된 데이터를 로드하여
사용자 사전과 비교를 진행하여 올바른 태그만 학습데이터로 생성

## 패키지 설치 및 Import

KoBERT 모델
* sentencepiece: Tokenizer 패키지
* seqeval: Evaluation 패키지
* transformers: KoBERT 패키지


In [ ]:
!pip install sentencepiece transformers seqeval tblib

In [ ]:
import pickle
import pandas as pd
from tqdm.notebook import tqdm

## 데이터 필터링 및 데이터셋 파일 생성

###  덤프 파일 로드

In [ ]:
filtered = pickle.load(open('./output/filtered.pkl', 'rb'))
error = pickle.load(open('./output/error.pkl', 'rb'))    
print(len(filtered), filtered[0][0])
print(len(error))

### 사용자 사전에 등록된 개체명만 추출

In [ ]:
import pandas as pd

# 기존의 사용자 사전 데이터 컬럼
# 아티스트(e, k), 배우(e, k), 멤버 -> PER
# 브랜드, 음원플랫폼, 방송플랫폼, 기획사, 팬덤(k,e), 기업 -> ORG
# 한국지명, 외국지명 -> LOC
# 앨범, 곡, 프로그램, 작품 -> AFW
# 수상, 수상2, 행사, 행사2 -> EVT
# 장르, 장르2 -> FLD
# 역할, 기타, 직업 -> 미사용

column_to_ner_tag = {
    '아티스트e': 'PER', '아티스트k': 'PER',
    '배우e': 'PER', '배우k': 'PER', 
    '멤버': 'PER',
    '브랜드': 'ORG', '음원플랫폼': 'ORG', 
    '방송플랫폼': 'ORG', '기획사': 'ORG', 
    '팬덤e': 'ORG', '팬덤k': 'ORG', 
    '기업': 'ORG',
    '한국지명': 'LOC', '외국지명': 'LOC',
    '앨범': 'AFW', '곡': 'AFW', 
    '프로그램': 'AFW', '작품': 'AFW',
    '행사': 'EVT', '행사2': 'EVT',
    '수상': 'EVT', '수상2': 'EVT', 
    '장르': 'FLD', '장르2': 'FLD',
    '기타': 'OTHER', '직업': 'OTHER',
    '역할': 'OTHER'
}
user_dict = {}
user_dict_df = pd.read_csv(filepath_or_buffer='./user-dic/tag_table.tsv', sep='\t', encoding='utf-8')
for col in user_dict_df.columns:
    tag = column_to_ner_tag[col]
    if tag == 'OTHER':
        continue
    for data in user_dict_df[col].dropna():
        user_dict[data.strip()] = tag

### NER 태그 중에서 사용자 사전에 정의된 태그만 필터링

ex) <블랙핑크:ORG>의 앨범 ... -> 블랙핑크 NER 태그가 인식되긴 했지만 PER이 아니므로 일단 필터링

주변 태그들을 확인하고 PER으로 교정할 수 있지만 수량이 많으므로 일단 스킵

In [ ]:
correct_dataset_string = []
correct_json = []

for _, json_data in tqdm(filtered):
    tag_str = ''
    prev_ner = 'O'
    prev_e_id = 0
    eojeol_tag_list = []
    filter_flag = True
    for morph in json_data['data']:
        if tag_str != '' and prev_e_id != morph['e_id']:
            tag_str += ' '
        if prev_e_id != morph['e_id']:
            prev_e_id = morph['e_id']
            eojeol_tag_list.append(morph['ner'])
        if morph['ner'] != 'O':
            if morph['ner'][-1] == 'B':
                tag_str = tag_str.strip()
                if tag_str in user_dict and user_dict[tag_str] != prev_ner:
                    filter_flag = False
                tag_str = morph['morph']
            else:
                tag_str += morph['morph']
        else:
            if tag_str != '':
                tag_str = tag_str.strip()
                if tag_str in user_dict and user_dict[tag_str] != prev_ner:
                    filter_flag = False
            tag_str = morph['morph']
            if tag_str in user_dict and user_dict[tag_str] != prev_ner:
                filter_flag = False
            tag_str = ''
    
    if prev_ner != 'O':
        prev_ner = morph['ner'][:-2]
    if filter_flag:
        correct_dataset_string.append(json_data['RawSentence'] + '\t' +
                                      ' '.join(eojeol_tag_list)
                                      )
        correct_json.append(json_data)
print(len(correct_dataset_string), len(filtered) - len(correct_dataset_string))

In [ ]:
# 필터링된 데이터를 DataFrame으로 변경
ner_tagged_df = pd.DataFrame(
    [x.split('\t') for x in correct_dataset_string],
    columns=['string', 'tag'] 
    )
ner_tagged_df

### Train/Test 데이터 분할

In [ ]:
train_idx = len(ner_tagged_df) // 11
shuffle_ner_tagged_df = ner_tagged_df.sample(frac=1).reset_index(drop=True)
with open('./ner_dataset/test.tsv', 'w', encoding='utf-8') as f:
    for _, row in list(shuffle_ner_tagged_df.iterrows())[:train_idx]:
        f.write(row['string'] + '\t' + row['tag'] + '\n')
with open('./ner_dataset/test.txt', 'w', encoding='utf-8') as f:
    for _, row in list(shuffle_ner_tagged_df.iterrows())[:train_idx]:
        f.write(row['string'] + '\n')
with open('./ner_dataset/train.tsv', 'w', encoding='utf-8') as f:
    for _, row in list(shuffle_ner_tagged_df.iterrows())[train_idx:]:
        f.write(row['string'] + '\t' + row['tag'] + '\n')
print(f'Train Dataset Size: {len(correct_dataset_string) - train_idx}')
print(f'Test Dataset Size : {train_idx}')

# KoBERT-NER 학습

## Train & Eval

Parameters
|Param|Desc|
|:----:|----|
|model_dir|저장된 모델 디렉토리(ner_model)|
|data_dir|데이터셋 디렉토리(ner_dataset)|
|train_batch_size|(Default: 32)|
|eval_batch_size|(Default: 64)|
|num_train_epochs|학습 Epoch(Default: 20)|
|do_train|training 모드|
|do_eval|eval 모드, training 모드와 함께 사용시 1,000 step 마다 Eval|

In [ ]:
!python main.py --data_dir=ner_dataset --model_dir=ner_model --do_train --do_eval --train_batch_size=16 --num_train_epochs=20

## Predict

Parameters
|Param|Desc|
|:----:|----|
|model_dir|저장된 모델 디렉토리(model)|
|input_file|Input 파일명|
|output_file|Output 파일명|
|batch_size|(Default: 32)|

In [ ]:
!python predict.py --model_dir=ner_model --input_file ./ner_dataset/test.txt